Import Packages<br>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import chi2
from scipy.stats import mode
import math
#print ("Leo_test")

Parameter setting
1. Folder path
2. Target field name
3. Target values
4. Index filed name

In [2]:
root_folder = 'HM1/'
train_file = 'train.csv'
#train_file = 'iris.csv'
test_file = 'test.csv'
Index = 'TransactionID'
Date = 'TransactionDT'
#Index = ''
Target = 'isFraud'
#Target = 'species'
#Target_value = 'versicolor' # 1
Target_value = 1

Read data

In [3]:
train = pd.read_csv(root_folder + train_file)
#test = pd.read_csv(root_folder + test_file)
#print (train.columns)
train.shape

(472432, 27)

Basic statistics <br>
How much missing values in each column

In [23]:
size = train.shape
Columns = train.columns.values
Columns = Columns[Columns != Index]
Columns = Columns[Columns != Date]
Categoric_features = ['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2']
#Categoric_features = []
Numeric_features = np.setdiff1d(Columns, Categoric_features)
MissingData = pd.DataFrame(index = Columns)
for i in Columns:
  #fig, ax  = plt.subplots(figsize= (5,3))
  #plt.hist(train[i])
  # if i in Numeric_features:
  #       identify = 'Numeric'
  # else:
  #   identify = 'Categoric'
  # plt.title(f"{i:}, {identify}")
  # if i == 'isFraud':
  #       continue
  tmp_df = train[(train[i]=='NotFound') | (train[i].isna())] #test[(test[i]=='NotFound') | (test[i].isna())]
  MissingData.loc[i,0] = tmp_df.shape[0]
Categoric_features.append('Num_Cat')
print (Numeric_features)
print (Categoric_features)
MissingData

['C1' 'C10' 'C11' 'C12' 'C13' 'C14' 'C2' 'C3' 'C4' 'C5' 'C6' 'C7' 'C8'
 'C9' 'TransactionAmt' 'isFraud']
['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'Num_Cat']


,0
ProductCD,0.0
card1,0.0
card2,7203.0
card3,1258.0
card4,1269.0
card5,3404.0
card6,1261.0
addr1,52428.0
addr2,52428.0
TransactionAmt,0.0


Dealing with missing data in <font color = 'red'> <b>train phase </font>


In [5]:
############# 1. remove missing records ############
train_filter = train.copy() 
############# 1. remove missing records ############


############# 2. Use most frequency values ############
train_Impu = train.copy() 
#test_Impu = test.copy()
############# 2. Use most frequency values ############

for i in tqdm(Columns):
  train_filter = train_filter[train_filter[i]!='NotFound']
  train_Impu.loc[train_Impu[i]=='NotFound',i] = train_Impu[i].value_counts().idxmax()
  # if i != 'isFraud':
  #   test_Impu[test_Impu[i]=='NotFound'] = test[i].value_counts().idxmax()
print (train_filter.shape[0]/size[0]," of data left in train_filter")


100%|██████████| 25/25 [00:02<00:00, 11.13it/s]

0.871657720052833  of data left in train_filter


<b>Information gain function<br></b>
| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Input_Dataframe</font> | pandas.DataFrame | This is your choosen subset of training data |
| <font color = 'green'> Split_feature </font>| String | The choose feature for calculating Information gain |
| <font color = 'green'> IG_methods</font> | String | 3 options, <font color = 'red'>Gini </font>: Gini index, <font color = 'red'>Entropy </font>: Max Entropy, <font color = 'red'>MisEr </font>: Miss classification|
| <font color = 'green'> Num_Cat </font>| String | 2 options, <font color = 'red'> Num </font>: Split feature is a numeric feature, <font color = 'red'> Cat </font>: Split feature is a categorical feature |




This function has 2 possible returns: <br>
&emsp;-- <font color = 'red'>[IG_feature, Split point]</font>: if <font color = 'green'>Split_feature</font> is a <b>numeric</b> feature<br>
&emsp;&emsp;-- <b>IG_feature</b>: single float variable representing <b>Information Gain</b> of <font color = 'green'>Split_feature</font>,<br> 
&emsp;&emsp;-- <b>Split point </b>: single float variable, the best binary split point for this numberic feature <font color = 'green'>Split_feature</font>,<br> 
&emsp;-- <font color = 'red'>[IG_feature, -3.14]</font>: if <font color = 'green'>Split_feature</font> is a<b> categorical </b>feature<br>
&emsp;&emsp;-- <b>IG_feature </b>: same as previous<br>
&emsp;&emsp;-- <b>-3.14 </b>: Meaningless, and not be used in the future. Just to make this function work recursively

In [6]:
def Info_Gain(Input_Dataframe, Split_feature, IG_methods, Num_Cat): #IG_methods = 'Gini'/ 'Entropy' / 'MisEr'

  length = len(Input_Dataframe)
  Fraud_arr = Input_Dataframe[Target]

  # propability of Fraud and not Fraud
  P_isF = np.sum(Fraud_arr == Target_value)/length
  P_notF = 1-P_isF

  Impu_root = 0
  Impu_leaves = [] #Impurity of each leaves
  Prop_leaves = [] #Propotion of each leaves
  IG_feature_Num = [] #IGs for numeric features

  ######### if Num_Cat == 'Mid' #########
  # This is used for recursive. 
  # When the Split_feature is a numeric feature, Info_Gain function will convert the numeric feature into a binary categorical feature (Line 74)
  # And then re-call the function Info_Gain(Input_Dataframe, Split_feature, IG_methods, 'Mid') , so the function will calculate the information gain for the numeric feature with the specific binary seperation
  ######### if Num_Cat == 'Mid' #########
  if Num_Cat == 'Cat' or Num_Cat == 'Mid':
    # Feature_classes: all unique classes, e.g. ['C','W','R', ...] for 'ProductCD'
    # P_unique_Fea: count for each classes, e.g. [100, 200, 100, ...]
    Feature_classes, P_unique_Fea = np.unique(Input_Dataframe[Split_feature], return_counts = True)

    if IG_methods == 'Gini':
      Impu_root = 1 - P_isF ** 2  - P_notF ** 2
      for i in Feature_classes:
        Sub_df = Input_Dataframe[Input_Dataframe[Split_feature] == i]
        Sub_len = len(Sub_df)
        Sub_P_isF = np.sum(Sub_df[Target] == Target_value)/Sub_len
        Sub_P_notF = 1-Sub_P_isF
        Impu_leaves.append(1 - Sub_P_isF ** 2  - Sub_P_notF ** 2)
        Prop_leaves.append(Sub_len/length)
    elif IG_methods == 'MisEr':
      Impu_root = 1 - max(P_isF,P_notF)
      for i in Feature_classes:
        Sub_df = Input_Dataframe[Input_Dataframe[Split_feature] == i]
        Sub_len = len(Sub_df)
        Sub_P_isF = np.sum(Sub_df[Target] == Target_value)/Sub_len
        Sub_P_notF = 1-Sub_P_isF
        Impu_leaves.append(1 - max(Sub_P_isF,Sub_P_notF))
        Prop_leaves.append(Sub_len/length)

    elif IG_methods == 'Entropy':
          
      ########## to deal with log(0) with 0 occurance ###########
      if P_isF == 0:
        Ent_P_isF = 0
      else:
        Ent_P_isF = P_isF * np.log2(P_isF)
      if P_notF == 0:
        Ent_P_notF = 0
      else:
        Ent_P_notF = P_notF * np.log2(P_notF)
      ########## to deal with log(0) with 0 occurance ###########

      Impu_root = -(Ent_P_notF + Ent_P_isF)
      for i in Feature_classes:
        Sub_df = Input_Dataframe[Input_Dataframe[Split_feature] == i]
        Sub_len = len(Sub_df)
        Sub_P_isF = np.sum(Sub_df[Target] == Target_value)/Sub_len
        Sub_P_notF = 1-Sub_P_isF
        if Sub_P_isF == 0:
          Ent_Sub_P_isF = 0
        else:
          Ent_Sub_P_isF = Sub_P_isF * np.log2(Sub_P_isF)
        if Sub_P_notF == 0:
          Ent_Sub_P_not_F = 0
        else:
          Ent_Sub_P_not_F = Sub_P_notF * np.log2(Sub_P_notF)
        Impu_leaves.append(-(Ent_Sub_P_isF + Ent_Sub_P_not_F))
        Prop_leaves.append(Sub_len/length)
    
  elif Num_Cat == 'Num':
    # this will automatically sort those unique values
    Sort_Input = np.unique(Input_Dataframe[Split_feature])
    Sort_Input = Sort_Input.astype(np.float32)

    # try every numeric values, add those information gain into the list "IG_feature_Num"   
    for i in Sort_Input:
      tmp_Input_Dataframe = Input_Dataframe.copy()
      # binary seperate the feature
      tmp_Input_Dataframe.loc[tmp_Input_Dataframe[Split_feature] <= i,Split_feature] = 0
      tmp_Input_Dataframe.loc[tmp_Input_Dataframe[Split_feature] > i,Split_feature] = 1
      IG_feature_Num.append(Info_Gain(tmp_Input_Dataframe,Split_feature, IG_methods, 'Mid'))

  Impu_root = np.array(Impu_root)
  Prop_leaves = np.array(Prop_leaves)

  # calculate the gain ratio if there are too much classes, this could reduce the breadth of the tree 
  if (Split_feature in Categoric_features) and (len(Feature_classes) > 200):
    P_unique_Fea = P_unique_Fea/sum(P_unique_Fea)
    log_P = np.log2(P_unique_Fea)
    SplitInfo = -np.matmul(P_unique_Fea,log_P.T)
    IG_feature =  float(f"{Impu_root - np.matmul(Impu_leaves,Prop_leaves.T):.5f}")
    IG_feature = IG_feature/SplitInfo
  # simplyt using the information gain
  elif (Split_feature in Categoric_features) or (Num_Cat == 'Mid'):
    IG_feature =  float(f"{Impu_root - np.matmul(Impu_leaves,Prop_leaves.T):.5f}")

  
  if Num_Cat == 'Num':
    IG_feature = max(IG_feature_Num)
    return [IG_feature, Sort_Input[IG_feature_Num.index(IG_feature)]] 
  elif Num_Cat == 'Mid':
    return IG_feature
  elif Num_Cat == 'Cat':
    return [IG_feature, -3.1415926] # -pi means it is a categorical feature
  

In [8]:
# demo of Info_Gain function
Info_Gain(train[0:4000], IG_methods= 'Gini', Split_feature='card1',Num_Cat='Cat')
Info_Gain(train[0:4000], IG_methods='Gini', Num_Cat='Num', Split_feature='C1')

[0.00028, 0.0]

<b>Chi-square test function<br></b>
| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Input_Dataframe</font> | pandas.DataFrame | This is your choosen subset of training data |
| <font color = 'green'> Split_feature </font>| String | The choose feature for calculating Chi_Square |
| <font color = 'green'> *Split_pts_Num</font> | float | This is an optional variable, it's only used in the function itself recursively|




This function has 2 possible returns: <br>
&emsp;-- <font color = 'red'>0</font>: The selected feature fails the Chi-test <br>
&emsp;-- <font color = 'red'>1</font>: The selected feature passes the Chi-test

To make sure that the tree can grow properly (increase possibility of growing to at least 2-depth) for small subset of the data (used for debuging the algorithm), user can set manually <font color = red>Chi-baisc = Chi-basic - 5</font>

In [9]:
def Chi_test(Input_Dataframe, Split_feature, *Split_pts_Num):
  length = Input_Dataframe.shape[0]
  Fraud_arr = Input_Dataframe[Target]
  # propability of Fraud and not Fraud
  P_isF = float(len(Fraud_arr[Fraud_arr == Target_value])/length)

  P_notF = 1-P_isF
  if (P_isF == 1) or (P_notF == 1):
    return 0
  Chi_sq = 0 # target split_feature's Chi square value
  alpha = 0.05 # your choose alpha value for the basic Chi-square values

  if Split_feature in Categoric_features:    
    Feature_classes = np.unique(Input_Dataframe[Split_feature])
    Degree_Freedom = (len(Feature_classes) - 1) * (2-1) # 2: T/F in isFraud
    for i in Feature_classes:
      Sub_df = Input_Dataframe[Input_Dataframe[Split_feature] == i]
      Sub_len = len(Sub_df)
      Sub_P_isF = len(Sub_df[Sub_df[Target] == Target_value])/Sub_len
      Sub_P_notF = 1-Sub_P_isF
      Chi_sq = Chi_sq + Sub_len * ((Sub_P_isF - P_isF)**2/P_isF + (Sub_P_notF - P_notF)**2/P_notF)
    Chi_basic = chi2.ppf(1 - alpha, Degree_Freedom) - 5
    
  elif Split_feature in Numeric_features:
    Degree_Freedom = 1*1
    tmp_Input_Dataframe = Input_Dataframe.copy()
    Split_pts_Num = Split_pts_Num
    # create a new field in dataframe "Num_Cat" to binary separate the feature.
    # notice that "Num_Cat" is also inside Categorical feature, check cell 4
    tmp_Input_Dataframe.loc[tmp_Input_Dataframe[Split_feature] <= Split_pts_Num,'Num_Cat'] = 0
    tmp_Input_Dataframe.loc[tmp_Input_Dataframe[Split_feature] > Split_pts_Num,'Num_Cat'] = 1
    Chi_sq = Chi_test(tmp_Input_Dataframe,'Num_Cat')
    Chi_basic = chi2.ppf(1 - alpha, Degree_Freedom) - 5
  
  if Split_feature != 'Num_Cat':
    if Chi_sq > Chi_basic:
      return 1  # good result, expanding
    else:
      return 0  # bad result, not expanding
  if Split_feature == 'Num_Cat':
      return Chi_sq

In [10]:
# demo of Chi_test
Chi_test(train[0:4000],'ProductCD')
Chi_test(train[0:4000],'C1', 50)

1

<b>Tree Node Structure<br></b>
| Node's property | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Self_Column</font> | String | Feature name for <b>the Node</b> |
| <font color = 'green'> Sub_branches </font>| list or float | <b>the Node</b>'s edges |
| <font color = 'green'> Sub_childs </font>| String | <b>the Node</b>'s children nodes |
| <font color = 'green'> Parent_Col</font>| String | <b>the Node</b>'s parent node's feature name |
| <font color = 'green'> Target</font> | float | <b>the Node</b>'s target|

<b>Noted:</b><br>
&emsp;-- For Categorical Features, Sub_branches equals to <b> the Node</b>'s unique values.  
&emsp;-- For Numerical Features, Sub_branches is a single float number (x0), which has the best Information Gain. <br>
&emsp;&emsp; Although <b> the Node</b>'s Sub_branches's length = 1, it has 2 Sub_childs. And the binary separation is done by the x0.<br>
&emsp;-- When a node has a child, it's Target is NaN.
<br>
| Node's Function | Usage demo | Definition | Output
| --- | --- | --- | --- |
| <font color = 'green'> is_leaf_node(self)</font> | Node.is_leaf_node() | Determine whther <b>the Node</b> is a leaf node. | 1 (leaf node) / 0 (not a leaf node)|
| <font color = 'green'> set_values(self, column, branch) </font>| Node.set_values(column = 'ProductCD', branch = ['W','R']) | Setting the property of <b>the Node</b> | No output|
| <font color = 'green'> set_Target(self, target) </font>| Node.set_Target(1) | Setting the property of <b>the Node</b> | No output|
| <font color = 'green'> add_child(self, node):</font>| Node.add_child(node = child_node)| link <b>the Node</b> with its child node |  No output|
| <font color = 'green'> set_parent(self, parent_col) | Node.set_parent('card1') | Setting the property of <b>the Node</b>  | No output|
| <font color = 'green'> get_target(self)</font>| Node.get_target() | Getting the Target value | 1 (Farud) / 0 (Not Fraud) / Nan (not a leaf node)|
| <font color = 'green'> get_column(self) </font>| Node.get_column() | Getting the property of <b>the Node</b> | <b>the Node</b>'s Feature name|
| <font color = 'green'> get_branch(self):</font>| Node.get_branch()| Getting all property of <b>the Node</b>|  list [] if the node represent a Categorical Feature / float number if the node is a Numerical Feature|
| <font color = 'green'> get_child(self):</font>| Node.get_child()| Getting all children nodes of <b>the Node</b>|  TreeNode() structure|

_________________________
________________________

<br>

<b>Build one Decision Tree<br></b>
| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Input_Dataframe</font> | pandas.DataFrame | The DataFrame of the Training data |
| <font color = 'green'> total_Col </font>| List | A list of usable features, defalut to be all features |
| <font color = 'green'> IG_methods</font> | String | Information Gain Method, 'Gini'/ 'Entropy' / 'MisEr', defalut to be 'Gini'|
| <font color = 'green'> depth</font> | float | The current depth's of the tree, default to be 0, only to be used inside the function for a recurssion purpose|

<br>
This function will return a root node of a tree.

In [11]:
class TreeNode:
    def __init__(self, Self_Column = None, Sub_branches = None, Target = None, Sub_childs = None, Parent_Col = None):
        self.Self_Column = Self_Column 
        self.Sub_branches = Sub_branches if Sub_branches is not None else []
        self.Target = Target
        self.Sub_childs = Sub_childs if Sub_childs is not None else []
        self.Parent_Col = Parent_Col

    def is_leaf_node(self):
        return self.Target is not None
    
    def set_values(self, column, branch):
        self.Self_Column = column
        self.Sub_branches = branch
    
    def set_Target(self, target):
        self.Target = target
    
    def add_child(self, node):
        self.Sub_childs.append(node)
        node.set_parent(self.Self_Column)

    def get_branch(self):
        return self.Sub_branches
    
    def get_child(self):
        return self.Sub_childs
    
    def get_column(self):
        return self.Self_Column
    
    def get_target(self):
        return self.Target
    
    def set_parent(self, parent_col):
        self.Parent_Col = parent_col
        

    
    
def build_DT(Input_Dataframe, total_Col = Columns, IG_methods = 'Gini', depth = 0):
    #IG_methods = 'Gini'/ 'Entropy' / 'MisEr'
    IG_method = IG_methods


    ############ can change the maximum depth of the tree here ###############
    max_depth = 5
    ############ can change the maximum depth of the tree here ###############


    Root_node = TreeNode()
    self_col = Input_Dataframe.columns.values
    depth = depth
    
    ################# find the best Information Gain feature ###################
    ################# and return the best split point if it is a numerical feature ###############
    IG_columns = pd.DataFrame(columns= ['Column', 'IG', 'SplitPts'])
    n = 0
    for i in self_col:
        #print (i)
        if i == Target or i == Index or not (i in total_Col):
            continue
        elif i in Categoric_features:
            [tmp_IG, Split_pts] = Info_Gain(Input_Dataframe, i, IG_method, 'Cat')
        elif i in Numeric_features:
            [tmp_IG, Split_pts] = Info_Gain(Input_Dataframe, i, IG_method, 'Num')
        #print (i, total_Col, 'i in total_Col', i in total_Col)
        IG_columns.loc[n,'Column'] = i
        IG_columns.loc[n,'IG'] = tmp_IG
        IG_columns.loc[n,'SplitPts'] = Split_pts
        n = n + 1
    ################# and return the best split point if it is a numerical feature ###############
    ################# find the best Information Gain feature ###################
    


    ############################ create a node for this selected feature ########################
    #print (IG_columns)
    Best_IG = IG_columns['IG'].max()
    [Best_Column, Split_Pts] = IG_columns.loc[IG_columns['IG'] == Best_IG, ['Column', 'SplitPts']].values[0]
    #print (Best_Column,Split_Pts)
    
    if Best_Column in Categoric_features:
        Root_node.set_values(Best_Column, np.unique(Input_Dataframe[Best_Column]))    
    elif Best_Column in Numeric_features:
        Root_node.set_values(Best_Column, Split_Pts)
    ############################ create a node for this selected feature ########################



    ############################ If the node failed the Chi-test, stop the tree ########################
    if (not Chi_test(Input_Dataframe, Best_Column, Split_Pts)):
        Root_node.set_Target(Input_Dataframe[Target].mode()[0])
        #print ('fail chi_test:', depth, Best_Column)
        #print ('max_depth', max_depth, depth == max_depth)
        return Root_node
    ############################ If the node failed the Chi-test, stop the tree ########################



    ############################ If the node failed the Chi-test, recursively expand the tree ########################
    else:
        depth = depth + 1

        #print ('pass chi_test:', depth)
        if depth < max_depth:
            #print ('in, Self_Col:', Root_node.get_column())
            #print ('in, Self_branch:', Root_node.get_branch())
            #print ('in, Self_branch_len:', len(Root_node.get_branch()))
            if (Root_node.Self_Column in Categoric_features) and (Root_node.Self_Column in total_Col):                
                for i in Root_node.Sub_branches:
                    #print ('Current Sub_Branch = ',i)
                    Sub_DF = Input_Dataframe[Input_Dataframe[Root_node.Self_Column] == i]    
                    tmp_Col = total_Col[total_Col != i]               
                    Root_node.add_child(build_DT(Sub_DF, tmp_Col, IG_method, depth))
                
                #print ("cat:", depth)
            elif (Root_node.Self_Column in Numeric_features) and (Root_node.Self_Column in total_Col):
                
                Sub_DF_left = Input_Dataframe[Input_Dataframe[Root_node.Self_Column] <= Root_node.Sub_branches]
                tmp_Col = total_Col[total_Col != i]
                Root_node.add_child(build_DT(Sub_DF_left, tmp_Col, IG_method, depth))
                Sub_DF_right = Input_Dataframe[Input_Dataframe[Root_node.Self_Column] > Root_node.Sub_branches]
                tmp_Col = total_Col[total_Col != i]
                Root_node.add_child(build_DT(Sub_DF_right, tmp_Col, IG_method, depth))
                
                
                #print ("num:", depth)
        else:
            #print ("depth maximum")
            Root_node.set_Target(Input_Dataframe[Target].mode()[0])
        return Root_node
    ############################ If the node failed the Chi-test, recursively expand the tree ########################


<b>DT_Predict :</b> Travelse the tree for prediction<br>
| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Input_Dataframe</font> | pandas.DataFrame | <b> One </b> record from the DataFrame of the testing/validation data |
| <font color = 'green'> Tree_node </font>| TreeNode() | The root node of one tree. |

<br>
This function will return a whether the targeted record is "Fraud" or "NotFraud".<br>

<b>Noted:</b><br>
In some cases, the trained tree has not seen a feature value from the testing/validation data, or the testing/validation data has a missing value. <br>
This function will return a list of all potential target values for the situations. <br>
Please refer to the <b>Section 1.2 in the report </b> for more details.

In [16]:
def DT_Predict(Input_Dataframe,Tree_node):
    tmp_node = Tree_node
    potential_predict = np.array([])
    list_TF = 0

    ################### DFS #######################
    while not tmp_node.is_leaf_node():
        tmp_col = tmp_node.get_column()
        #print (tmp_col)
        #tmp_col = tmp_node.Self_Column
        #print (Input_Dataframe[tmp_col])
        data_val = Input_Dataframe[tmp_col].values[0]
        branches = tmp_node.get_branch()
        #branches = tmp_node.Sub_branches
        
        if tmp_col in Categoric_features:
            #print ("cat:", tmp_col)
            #print (data_val)
            sub_node_idx = np.where(branches == data_val)[0]
            if sub_node_idx.shape[0] > 0:
                sub_node_idx = sub_node_idx[0]
                list_TF = 0
            else:
                sub_node_idx = -1
                list_TF = 1
                
        elif tmp_col in Numeric_features:
            list_TF = 0
            #print ("num:", tmp_col)
            if data_val <= branches:
                sub_node_idx = 0
            else:
                sub_node_idx = 1
        
        child_nodes = tmp_node.get_child()
        #print (sub_node_idx)
        #print (child_nodes)

        ########################### when the tree has not seen the value or it is a missing data ##########################
        if sub_node_idx == -1:
            for i in branches:
                tmp_Dataframe = Input_Dataframe.copy()
                sub_node_idx = np.where(branches == i)[0][0]
                tmp_node = child_nodes[sub_node_idx]
                tmp_Dataframe[tmp_col] = i
                potential_predict = np.append(potential_predict, DT_Predict(tmp_Dataframe, tmp_node))
        ########################### when the tree has not seen the value or it is a missing data ##########################
        
        else:
            #print (sub_node_idx)
            #print (tmp_node.get_column(), len(tmp_node.get_child()), tmp_node.is_leaf_node(), tmp_node.get_target())
            tmp_node = child_nodes[sub_node_idx]
    #print (sub_node_idx)
    ################### DFS #######################


    if list_TF == 1:
        return potential_predict
    else:
        return tmp_node.Target


<b>tree_df :</b> Saving the tree to a DataFrame<br>
| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> root_node </font>| TreeNode() | The root node of one tree. |

<br>
This function will return a DataFrame to store the tree.<br>
More details are included in the report.
<br>
<br>

<b>Predict_df :</b> Using the dataframe to do a prediction<br>

| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Input_Dataframe</font> | pandas.DataFrame | <b> One </b> record from the DataFrame of the testing/validation data |
| <font color = 'green'> tree_df </font>| pandas.DataFrame | The DataFrame represening the tree |
| <font color = 'green'> current_index </font>| int | The index from the DataFrame, only used for recursive purpose inside the function itself |

<br>
The basic logic of this function is the same as <b>DT_Predict </b>. <br>
The only difference is that this function is using <b> current_index </b> to track done the current node.

In [13]:
def tree_df(root_node):

    queue = [(root_node,0,0)]
    list_node = []
    while queue:
        current_node, depth, sub_branch = queue.pop(0)
        current_col_name = current_node.get_column()
        current_branches = current_node.get_branch()
        current_childs = current_node.get_child()
        childs_name = []
        for i in range(len(current_childs)):
            childs_name.append(current_childs[i].get_column())
        list_node.append((current_col_name,current_branches,childs_name,depth, current_node.get_target(),current_node.Parent_Col,sub_branch))
        #print (list_node)
        tmp_child = current_node.get_child()
        if len(tmp_child) > 0:
            if current_col_name in Categoric_features:
                for i in range(len(tmp_child)):
                    queue.append((tmp_child[i],depth+1,current_branches[i]))
            elif current_col_name in Numeric_features:
                queue.append((tmp_child[0],depth+1,current_branches))
                queue.append((tmp_child[1],depth+1,current_branches+1))

    tree_df = pd.DataFrame(list_node,columns = ['Column_name','Branches','Child_Columns','Depth','Target','Parent_Col','Parent_Branch'])
    return tree_df




def Predict_df(Input_Dataframe,tree_df, current_index = 0):

    potential_predict = np.array([])
    list_TF = 0
    current_index = current_index
    while pd.isna(tree_df.loc[current_index,'Target']):
        
        tmp_col = tree_df.loc[current_index,'Column_name']
        data_val = Input_Dataframe[tmp_col].values[0]
        branches = tree_df.loc[current_index,'Branches']
        current_depth = tree_df.loc[current_index,'Depth']
        if tmp_col in Categoric_features:
            #print ("cat:", tmp_col)
            #print (data_val)
            sub_node_idx = np.where(branches == data_val)[0]
            if sub_node_idx.shape[0] > 0:
                sub_node_idx = sub_node_idx[0]
                tmp_branch = branches[sub_node_idx]
                list_TF = 0
            else:
                sub_node_idx = -1
                list_TF = 1
                
        elif tmp_col in Numeric_features:
            list_TF = 0
            #print ("num:", tmp_col)
            if data_val <= branches:
                sub_node_idx = 0
                tmp_branch = branches
            else:
                sub_node_idx = 1
                tmp_branch = branches + 1
        
        child_nodes_name = tree_df.loc[current_index,'Child_Columns']

        if sub_node_idx == -1:
            for i in branches:
                tmp_Dataframe = Input_Dataframe.copy()
                #print ('i=', i)
                #print (branches,np.where(branches == i)[0])
                branch_idx = np.where(branches == i)[0][0]
                
                Child_Name = child_nodes_name[branch_idx]
                
                current_index = tree_df[(tree_df['Column_name'] == Child_Name) & (tree_df['Parent_Col'] == tmp_col) & (tree_df['Parent_Branch'] == i) & (tree_df['Depth'] == current_depth + 1)].index[0]
                
                tmp_Dataframe[tmp_col] = i
                potential_predict = np.append(potential_predict, Predict_df(tmp_Dataframe, tree_df, current_index))
        else:
            #print (tmp_node.get_column(), len(tmp_node.get_child()), tmp_node.is_leaf_node(), tmp_node.get_target())

            current_index = tree_df[(tree_df['Column_name'] == child_nodes_name[sub_node_idx]) & (tree_df['Parent_Col'] == tmp_col) & (tree_df['Parent_Branch'] == tmp_branch) & (tree_df['Depth'] == current_depth + 1)].index[0]
    #print (sub_node_idx)
    if list_TF == 1:
        return potential_predict
    else:
        return tree_df.loc[current_index,'Target']

#tree_df(tree_node)

<b>Build Random Forest </b><br>
Using <b>DT_Predict</b> function <br>
In line 57, some other version may require you remove the index, the optional line is added in line 58

In [ ]:
# How many trees inside the random forest 
Tree_no = 3

All_predict = []
All_observe = []


for ii in range(Tree_no):
    # shuffle the train.csv dataset so that each tree is using different subset
    # this is using train_filter, you can also use train_Impu
    train_filter = train_filter.sample(frac = 1)
    # split the train.csv dataset into "positive" and "negative" based on "Fraud" or "not Fruad"
    positive = train_filter[train_filter['isFraud'] == 1]
    negative = train_filter[train_filter['isFraud'] == 0]
    pos_size = positive.shape
    neg_size = negative.shape

    
    # 0.002: 0.2% of positive and negative into test 
    test_positive = positive[0:int(pos_size[0]*0.002)] 
    test_negative = negative[0:int(neg_size[0]*0.002)]

    # [0.002: 1/100]: 0.8% of positive and negative into training
    train_positive = positive[int(pos_size[0]*0.002):int(pos_size[0]/100)]
    train_negative = negative[int(neg_size[0]*0.002):int(neg_size[0]/100)]

    # create the pool for training and validation
    pool_train = pd.concat([train_positive, train_negative])
    pool_test = pd.concat([test_positive, test_negative])
    pool_test = pool_test.sample(frac = 1)
    exp_test = pool_test[0:int(len(pool_test))]

    # randomly select 20 features for each tree
    np.random.shuffle(Columns)
    tmp_total_Col = Columns[0:20] # change 20 to select more or less columns

    pool_train = pool_train.sample(frac = 1)
    exp_train = pool_train[0:int(len(pool_train)*0.99)]
    print (exp_train.shape)

    observe = []
    predict = [] 
    #IG_methods = 'Gini'/ 'Entropy' / 'MisEr'
    tree_node = build_DT(exp_train, total_Col= tmp_total_Col, IG_methods = 'Gini')
    T_Df = tree_df(tree_node)
    T_Df.to_pickle('Tree_Gini_Filt' + str(ii) + '.pkl')
    
    print (str(ii) + 'th DT')
    for i in tqdm(range(exp_test.shape[0])):
        tmp_dataframe = pd.DataFrame(exp_test.iloc[i]).T
        #(type(tmp_dataframe))
        #print (tmp_dataframe)
        target_predict = DT_Predict(tmp_dataframe,tree_node)
        if not isinstance(target_predict, np.ndarray):
            predict.append(target_predict)
        else:
            predict.append(mode(target_predict).mode[0])
            #predict.append(mode(target_predict).mode)
    observe = exp_test[Target]


    ################### Calculate the Confuse Matrix and Balanced Acc #####################
    All_predict.append(predict)
    All_observe.append(observe)

    Confuse_Matrix = pd.DataFrame(0,columns = ['Predict_T','Predict_F'],index= ['Observe_T','Observe_F'])
    for i in tqdm(range(len(predict))):
        if predict[i] == 1 and observe.iloc[i] == 1:
            Confuse_Matrix.loc['Observe_T','Predict_T'] = Confuse_Matrix.loc['Observe_T','Predict_T'] + 1
        elif predict[i] == 1 and observe.iloc[i] == 0:
            Confuse_Matrix.loc['Observe_F','Predict_T'] = Confuse_Matrix.loc['Observe_F','Predict_T'] + 1
        elif predict[i] == 0 and observe.iloc[i] == 1:
            Confuse_Matrix.loc['Observe_T','Predict_F'] = Confuse_Matrix.loc['Observe_T','Predict_F'] + 1
        elif predict[i] == 0 and observe.iloc[i] == 0:
            Confuse_Matrix.loc['Observe_F','Predict_F'] = Confuse_Matrix.loc['Observe_F','Predict_F'] + 1
    Recall_P = Confuse_Matrix.loc['Observe_T','Predict_T'] / sum(Confuse_Matrix.loc['Observe_T'])
    Recall_F = Confuse_Matrix.loc['Observe_F','Predict_F'] / sum(Confuse_Matrix.loc['Observe_F'])
    Balanced_Acc = (Recall_P + Recall_F)/2
    print("Balanced_Acc:", Balanced_Acc)
    Confuse_Matrix.to_csv('Conf_Mat_Gini_Filt' + str(ii))
    ################### Calculate the Confuse Matrix and Balanced Acc #####################

(3261, 27)
0th DT


  0%|          | 0/823 [00:00<?, ?it/s]C:\Users\cgirlamo\AppData\Local\Temp\ipykernel_3144\1479823888.py:57: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  predict.append(mode(target_predict).mode[0])
100%|██████████| 823/823 [00:00<00:00, 9073.57it/s]


Balanced_Acc: 0.49937733499377335
(3261, 27)
1th DT


  0%|          | 0/823 [00:00<?, ?it/s]C:\Users\cgirlamo\AppData\Local\Temp\ipykernel_3144\1479823888.py:57: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  predict.append(mode(target_predict).mode[0])
100%|██████████| 823/823 [00:00<00:00, 9371.64it/s]


Balanced_Acc: 0.4987546699875467
(3261, 27)
2th DT


100%|██████████| 823/823 [00:00<00:00, 9706.83it/s]

Balanced_Acc: 0.49937733499377335


In [1]:
All_predict

NameError: name 'All_predict' is not defined

In [ ]:
T_Df

,Column_name,Branches,Child_Columns,Depth,Target,Parent_Col,Parent_Branch
0,card3,"[106.0, 117.0, 143.0, 144.0, 146.0, 147.0, 150...","[ProductCD, ProductCD, ProductCD, ProductCD, P...",0,NaN,None,0
1,ProductCD,[C],[],1,0.0,card3,106.0
2,ProductCD,[R],[],1,0.0,card3,117.0
3,ProductCD,[W],[],1,0.0,card3,143.0
4,ProductCD,[H],[],1,1.0,card3,144.0
5,ProductCD,[W],[],1,0.0,card3,146.0
6,ProductCD,[C],[],1,0.0,card3,147.0
7,C13,0.0,"[card2, C10]",1,NaN,card3,150.0
8,ProductCD,[H],[],1,1.0,card3,181.0
9,ProductCD,[C],[],1,0.0,card3,185.0


<b>Predict_np :</b> Using a numpy.Array to do a prediction<br>

| Input Variable | Type | Definition |
| --- | --- | --- |
| <font color = 'green'> Input_np</font> | pandas.DataFrame | <b> One </b> record from the Array of the testing/validation data |
| <font color = 'green'> tree_np </font>| pandas.DataFrame | The Array represening the tree |
| <font color = 'green'> current_index </font>| int | The index from the DataFrame, only used for recursive purpose inside the function itself |

<br>
The basic logic of this function is the same as <b>Predict_df </b>. <br>
The only difference is that this function is using column index instead of column (feature name) itself.

In [36]:
Columns = train.columns.values
Columns = Columns[Columns != Index]
Columns = Columns[Columns != Date]
Categoric_features_index = np.where(np.isin(Columns,Categoric_features))[0]
Numeric_features_index = np.where(np.isin(Columns,Numeric_features))[0]


def Predict_np(Input_np, tree_np, current_index = 0):
                       
    potential_predict = np.array([])
    list_TF = 0
    #current_index = current_index
    while np.isnan(tree_np[current_index][4]): #[4]:"Target"
        
        tmp_col = tree_np[current_index][0] # [0]: 'Column_name'
        #print (tmp_col)
        tmp_col_index = np.where(Columns == tmp_col)[0][0] 
        #print (tmp_col_index)
        data_val = Input_np[tmp_col_index]
        branches = tree_np[current_index][1] # [1]:'Branches'
        current_depth = tree_np[current_index][3]
        if tmp_col_index in Categoric_features_index:
            #print ("cat:", tmp_col)
            #print (data_val)
            sub_node_idx = np.where(branches == data_val)[0]
            if sub_node_idx.shape[0] > 0:
                sub_node_idx = sub_node_idx[0]
                tmp_branch = branches[sub_node_idx]
                list_TF = 0
            else:
                sub_node_idx = -1
                list_TF = 1
                
        elif tmp_col_index in Numeric_features_index:
            list_TF = 0
            #print ("num:", tmp_col)
            if data_val <= branches:
                sub_node_idx = 0
                tmp_branch = branches
            else:
                sub_node_idx = 1
                tmp_branch = branches + 1
        
        child_nodes_name = tree_np[current_index][2] #[2]: "Child_Columns"
        
        if sub_node_idx == -1:
            len_branches = len(branches)
            branch_index_pool = np.arange(len_branches)
            np.random.shuffle(branch_index_pool)
            n = 0
            #tmp_np = np.array()
            #print (tmp_col, len(branches))
            for i in branch_index_pool:
                tmp_np = Input_np.copy()
                #print ('i=', i)
                #print (branches,np.where(branches == i)[0])
                branch_idx = i
                n = n + 1
                if len_branches > 1000 & n > len_branches/20:
                    break
                Child_Name = child_nodes_name[branch_idx]
                current_index = np.where((tree_np[:,0] == Child_Name) & (tree_np[:,5] ==tmp_col) & (tree_np[:,6] == branches[branch_idx]) & (tree_np[:, 3] == current_depth + 1))[0][0]
                #current_index = tree_df[(tree_df['Column_name'] == Child_Name) & (tree_df['Parent_Col'] == tmp_col) & (tree_df['Parent_Branch'] == i)].index[0]
                
                tmp_np[tmp_col_index] = i
                potential_predict = np.append(potential_predict, Predict_np(tmp_np, tree_np, current_index))
        else:
            #print (tmp_node.get_column(), len(tmp_node.get_child()), tmp_node.is_leaf_node(), tmp_node.get_target())
            current_index = np.where((tree_np[:,0] == child_nodes_name[sub_node_idx]) & (tree_np[:,5] ==tmp_col) & (tree_np[:,6] == tmp_branch) & (tree_np[:, 3] == current_depth + 1))[0][0]
            
    #print (sub_node_idx)
    if list_TF == 1:
        return potential_predict
    else:
        return tree_np[current_index][4]

Demo of using <b>Predict_df</b> and <b>Predict_np</b><br>

In [42]:
observe = -1
predict_tree = -1
predict_df = -1
predict_np = -1
#IG_methods = 'Gini'/ 'Entropy' / 'MisEr'

i = 50 # change it to your desired record index
tmp_dataframe = pd.DataFrame(exp_test.iloc[i]).T
observe = tmp_dataframe[Target].values

######## using DT_Predict (trees) ############
target_predict = DT_Predict(tmp_dataframe,tree_node)
if not isinstance(target_predict, np.ndarray):
    predict_tree = target_predict
else:
    predict_tree = mode(target_predict).mode[0] # may have error for different mode version. Can switch to line 14
    #predict_tree = mode(target_predict).mode
######## using DT_Predict (trees) ############



######## using Predict_df (DF) ############
target_predict = Predict_df(tmp_dataframe,T_Df)
if not isinstance(target_predict, np.ndarray):
    predict_df = target_predict
else:
    predict_df = mode(target_predict).mode[0]
    #predict_df = mode(target_predict).mode
######## using Predict_df (DF) ############



######## using Predict_np (np.Array) ############
T_np = np.array(T_Df)
tmp_dataframe_copy = tmp_dataframe.copy()
tmp_dataframe_copy = tmp_dataframe_copy.drop(columns = [Index,Date])
tmp_np = np.array(tmp_dataframe_copy)[0]

target_predict = Predict_np(tmp_np,T_np)
if not isinstance(target_predict, np.ndarray):
    predict_np = target_predict
else:
    predict_np = mode(target_predict).mode[0]
    #predict_np = mode(target_predict).mode
######## using Predict_np (np.Array) ############


print ('obser:', observe)
print ('[Tree, DF, NP]: [', predict_tree, ',', predict_df,',', predict_np, ']')

obser: [0]
[Tree, DF, NP]: [ 0 , 0.0 , 0.0 ]
